<a href="https://colab.research.google.com/github/nbadiuc/Automatic-Text-Simplification-Based-On-NER/blob/main/Thesis_Automatic_Text_Simplification_Based_On_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installations and imports

In [ ]:
!pip install torch
!pip install transformers
!pip install xformers
!pip install sentencepiece
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from transformers import pipeline

import torch
from sklearn.metrics.pairwise import cosine_distances
import pandas as pd
import zipfile
import os
import string
import requests
import json
import random
import re

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# The baseline model

In [ ]:
baseline_tokenizer = AutoTokenizer.from_pretrained("orzhan/rugpt3-simplify-large", padding_side='left')
baseline_model = AutoModelForCausalLM.from_pretrained("orzhan/rugpt3-simplify-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
baseline_simplifier = pipeline("text-generation", model=baseline_model, tokenizer=baseline_tokenizer, max_length=160)

In [ ]:
def base_model(sentence):
    sentence = "<s>" + sentence + "<Simplify:>"
    while sentence.find("EndText") == -1:
        sentence = baseline_simplifier(sentence)[0]['generated_text']
    return sentence[sentence.find("<Simplify:>")+12:sentence.find("EndText")-1]

In [ ]:
print(base_model("Финансовым директором социальной сети Facebook назначен 39-летний Дэвид Эберсман (David Ebersman), сообщает The Wall Street Journal."))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Дэвид Эберсман (David Ebersman) - финансовый директор социальной сети Facebook.


#BERT and synonym dictionary models

In [ ]:
# download the dataset into the /tmp folder
!wget "https://github.com/nerel-ds/NEREL/archive/refs/heads/master.zip" -O "/tmp/nerel.zip"

zip_ref = zipfile.ZipFile('/tmp/nerel.zip', 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

--2023-06-07 12:26:38--  https://github.com/nerel-ds/NEREL/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/nerel-ds/NEREL/zip/refs/heads/master [following]
--2023-06-07 12:26:38--  https://codeload.github.com/nerel-ds/NEREL/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/nerel.zip’

/tmp/nerel.zip          [    <=>             ]   6.14M  8.38MB/s    in 0.7s    

2023-06-07 12:26:39 (8.38 MB/s) - ‘/tmp/nerel.zip’ saved [6443546]



In [ ]:
# reading a text and its annotation
def read_text(text_location="/tmp/NEREL-master/NEREL-v1.1/train/023.txt"):

    with open(text_location) as f:
        lines = f.readlines()
    
    # getting rid of the titles of the articles - they are not full sentences
    offset = len(lines[0]) # title length

    text = ''
    for i in range(1, len(lines)):
        text += lines[i]

    ann_location = text_location[:-3] + "ann"
    with open(ann_location) as f:
        ann = f.readlines()

    return text, ann, offset

In [ ]:
text, ann, offset = read_text(text_location="/tmp/NEREL-master/NEREL-v1.1/train/106106_text.txt")
print("Text:")
print(text)
print("Annotation:")
print(ann)
print("Offset:")
print(offset)

Text:

Эдуард Назаров
11 сентября 2016 года на 75 году жизни в Москве скончался советский и российский мультипликатор, режиссёр и художник, народный артист Российской Федерации, автор известного мультфильма «Жил-был пёс» Эдуард Назаров.

Эдуард Васильевич он давно болел и имел проблемы с сосудами.

Время и место похорон пока уточняются.

Эдуард Назаров родился 23 ноября 1941 года в Москве. Закончил Строгановское художественно-промышленное училище. Работал в мультипликации с 1959 года сначала прорисовщиком, затем ассистентом художника-постановщика у Михаила Цехановского, художником-постановщиком у Фёдора Хитрука, режиссёром.

Занимался оформлением книг и журналов: «Квант», «Весёлые картинки», «Мурзилка», «Химия и жизнь», «Знание — сила», «Миша» и других.

C 1979 по 2000 год преподавал на Высших курсах сценаристов и режиссёров.

В 1993 году организовал школу-студию мультипликаторов-режиссёров «ШАР» совместно с Андреем Хржановским, Фёдором Хитруком, Юрием Норштейном.

Соавтор и ведущий те

In [ ]:
# using annotation, generates a mask that contains 1s in the places corresponding to named entities and 0s otherwise
def generate_NE_mask(text, ann, offset):
    mask = [0] * len(text)
    for line in ann:
        if line[0] == 'T': # "T" - terms; in annotation, they correspond to named entites
            if ';' in line:
                line = line.split(';')[0]
            start_ind, end_ind = line.split('\t')[1].split(' ')[1:3]
            #named_entity = line.split('\t')[2].strip()
            start_ind = int(start_ind) - offset
            end_ind = int(end_ind) - offset
            if start_ind >= 0:
                for i in range(start_ind, end_ind):
                    mask[i] = 1
    return mask

In [ ]:
mask = generate_NE_mask(text, ann, offset)
for i in range(min(len(mask), 250)):
    print(mask[i], end='')
print()
print(text[:min(len(mask), 250)])

0111111111111110111111111111111111111011111111110000000001111110111111111011111111100011111111110111111111111110011111111000111111110011111111111111111111111111111111111100000000000000000000000000000000111111111110011111111111111000111111111111111110

Эдуард Назаров
11 сентября 2016 года на 75 году жизни в Москве скончался советский и российский мультипликатор, режиссёр и художник, народный артист Российской Федерации, автор известного мультфильма «Жил-был пёс» Эдуард Назаров.

Эдуард Васильевич 


In [ ]:
# splits text into sentences remembering start positions of each one
# (relative to text that does not contain name of the article, so, to match the positions in annotation, you need to add offset)
def split_text_into_sentences(text):
    tokenized_sentences = sent_tokenize(text, language="russian")
    sentences = []
    sentence_start_positions = []
    for i in range(len(tokenized_sentences)):
        tokenized_sentences[i] = tokenized_sentences[i].strip()
        if len(tokenized_sentences[i]) != 0:
            sentences.append(tokenized_sentences[i].replace('\n', ' '))
            sentence_start_positions.append(text.find(tokenized_sentences[i]))
    return sentences, sentence_start_positions

In [ ]:
sentences, sentence_start_positions = split_text_into_sentences(text)
for i in range(len(sentences)):
    print(sentence_start_positions[i], sentences[i])

1 Эдуард Назаров 11 сентября 2016 года на 75 году жизни в Москве скончался советский и российский мультипликатор, режиссёр и художник, народный артист Российской Федерации, автор известного мультфильма «Жил-был пёс» Эдуард Назаров.
232 Эдуард Васильевич он давно болел и имел проблемы с сосудами.
294 Время и место похорон пока уточняются.
334 Эдуард Назаров родился 23 ноября 1941 года в Москве.
387 Закончил Строгановское художественно-промышленное училище.
446 Работал в мультипликации с 1959 года сначала прорисовщиком, затем ассистентом художника-постановщика у Михаила Цехановского, художником-постановщиком у Фёдора Хитрука, режиссёром.
627 Занимался оформлением книг и журналов: «Квант», «Весёлые картинки», «Мурзилка», «Химия и жизнь», «Знание — сила», «Миша» и других.
759 C 1979 по 2000 год преподавал на Высших курсах сценаристов и режиссёров.
833 В 1993 году организовал школу-студию мультипликаторов-режиссёров «ШАР» совместно с Андреем Хржановским, Фёдором Хитруком, Юрием Норштейном.


In [ ]:
# splits sentences into words remembering start position of each one (relative to text without the name of article)
# gets rid of punctuation and stop words while doing so
def split_sentences_into_words(text, sentences, sentence_start_positions):

    words = []
    words_start_positions = []

    for i in range(len(sentences)):
        current_words = word_tokenize(sentences[i], language="russian")
        stop = set(stopwords.words('russian') + list(string.punctuation))
        current_words = [word for word in current_words if word not in stop]

        current_piece_of_text = text[sentence_start_positions[i]:]
        current_words_start_positions = []
        for word in current_words:
            current_words_start_positions.append(current_piece_of_text.find(word) + sentence_start_positions[i])
        words.append(current_words)
        words_start_positions.append(current_words_start_positions)

    return words, words_start_positions

In [ ]:
words, words_start_positions = split_sentences_into_words(text, sentences, sentence_start_positions)
print(words)
print(words_start_positions)

[['Эдуард', 'Назаров', '11', 'сентября', '2016', 'года', '75', 'году', 'жизни', 'Москве', 'скончался', 'советский', 'российский', 'мультипликатор', 'режиссёр', 'художник', 'народный', 'артист', 'Российской', 'Федерации', 'автор', 'известного', 'мультфильма', '«', 'Жил-был', 'пёс', '»', 'Эдуард', 'Назаров'], ['Эдуард', 'Васильевич', 'давно', 'болел', 'имел', 'проблемы', 'сосудами'], ['Время', 'место', 'похорон', 'пока', 'уточняются'], ['Эдуард', 'Назаров', 'родился', '23', 'ноября', '1941', 'года', 'Москве'], ['Закончил', 'Строгановское', 'художественно-промышленное', 'училище'], ['Работал', 'мультипликации', '1959', 'года', 'сначала', 'прорисовщиком', 'затем', 'ассистентом', 'художника-постановщика', 'Михаила', 'Цехановского', 'художником-постановщиком', 'Фёдора', 'Хитрука', 'режиссёром'], ['Занимался', 'оформлением', 'книг', 'журналов', '«', 'Квант', '»', '«', 'Весёлые', 'картинки', '»', '«', 'Мурзилка', '»', '«', 'Химия', 'жизнь', '»', '«', 'Знание', '—', 'сила', '»', '«', 'Миша', '»

In [ ]:
word_frequencies = pd.read_csv('/content/drive/My Drive/freqrnc2011.csv', sep='\t', index_col='Lemma')

len_threshold = 7
frequency_threshold = 1/31

def word_complexity(word, method):
    if method == "len":
        return len(word)
    if method == "frequency":
        if word in word_frequencies.index:
            word_frequency = word_frequencies.loc[word]['Freq(ipm)']
            if isinstance(word_frequency, pd.core.series.Series):
                word_frequency = max(word_frequency)
            return 1/word_frequency
        else:
            return 1/(frequency_threshold - 1)
#            return 15/word_frequency + len(word)/14
#        else:
#            return len(word)/7

In [ ]:
def get_normal_form(word):
    return morph.parse(word)[0].normal_form

In [ ]:
bert_simplifier = pipeline("fill-mask", model="bert-base-multilingual-cased")

def bert_model(initial_sentence, masked_word, word_start_position_in_initial_sentence, complexity_method):
    masked_sentence = initial_sentence[:word_start_position_in_initial_sentence] + "[MASK]" + initial_sentence[word_start_position_in_initial_sentence + len(masked_word):]
    results = bert_simplifier(masked_sentence)
    for result in results:
        suggested_word = result['token_str']
        if word_complexity(get_normal_form(suggested_word), complexity_method) < word_complexity(masked_word, complexity_method):
            return suggested_word
    return masked_word 

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def get_synonyms(word):
    url = 'https://dictionary.yandex.net/api/v1/dicservice.json/lookup'
    params = {
        'key': 'dict.1.1.20230528T133010Z.158a1e5336183ca0.4e03e68250b1dfaa01b77acbf3439b0e45d1a614',
        'lang': 'ru-ru',
        'text': word
    }
    response = requests.get(url, params=params)
    data = json.loads(response.text)
    try:
        synonyms_list = []
        for i in data['def'][0]['tr']:
            for j in i['syn']:
                synonyms_list.append(j['text'])
            synonyms_list.append(i['text'])
        return synonyms_list
    except:
        return []

In [ ]:
def get_declension(word):
    p = morph.parse(word)[0]
    tags = set()
    for tag in {p.tag.case, p.tag.gender, p.tag.number, p.tag.tense, p.tag.person, p.tag.aspect, p.tag.involvement, p.tag.mood, p.tag.voice}:
        if tag != None:
            tags.add(tag)
    return tags

In [ ]:
get_declension("красноречивый")

{'masc', 'nomn', 'sing'}

In [ ]:
def decline(word, tags):
    p = morph.parse(word)[0]
    for tag in tags:
        if p.inflect({tag}) != None:
            p = p.inflect({tag})
    return p.word

In [ ]:
def synonym_model(word, complexity_method):
    declension = get_declension(word)
    normalized_word = get_normal_form(word)
    synonyms = get_synonyms(normalized_word)
    result = word
    for synonym in synonyms:
        if word_complexity(synonym, complexity_method) < word_complexity(result, complexity_method):
            result = synonym
    result = decline(result, declension)
    if word[0].isupper():
        result = result.capitalize()
    return result

In [ ]:
synonym_model("апробации", "frequency")

'апробации'

In [ ]:
simple_dir = '/content/drive/My Drive/simplifications'
if not os.path.exists(os.path.join(simple_dir, 'original')):
    os.makedirs(os.path.join(simple_dir, 'original'))

def write_list_to_file(list, path): # path where the file is located
    with open(path, 'w') as f:
        for item in list:
            f.write("%s\n" % item)

def read_list_from_file(path):
    with open(path) as f:
        sentences = f.readlines()
    return sentences

In [ ]:
def simplify(text_location, model, complexity_method):

    text, ann, offset = read_text(text_location)
    mask = generate_NE_mask(text, ann, offset)
    sentences, sentence_start_positions = split_text_into_sentences(text)
    words, words_start_positions = split_sentences_into_words(text, sentences, sentence_start_positions)

    file_name = text_location.split('/')[-1]
    if not os.path.exists(os.path.join(simple_dir, 'original', file_name)):
        write_list_to_file(sentences, os.path.join(simple_dir, 'original', file_name))
    simplified_sentences = []
    
    for sentence_ind in range(len(sentences)):
        if model == "baseline":
            simplified_sentence = base_model(sentences[sentence_ind])
        else:
            simplified_sentence = sentences[sentence_ind]
            words_start_positions_in_simplified_sentence = words_start_positions[sentence_ind][:]
            for word_ind in range(len(words[sentence_ind])):
                words_start_positions_in_simplified_sentence[word_ind] -= sentence_start_positions[sentence_ind]

            for word_ind in range(len(words[sentence_ind])):
                current_word = words[sentence_ind][word_ind]
                if mask[words_start_positions[sentence_ind][word_ind]] == 0: # if the current word is not part of a named entity
                    normalized_current_word = get_normal_form(current_word)
                    if complexity_method == "len":
                        threshold = len_threshold
                    else:
                        threshold = frequency_threshold
                    if word_complexity(normalized_current_word, complexity_method) > threshold:
                        current_word_start_position_in_initial_sentence = words_start_positions[sentence_ind][word_ind] - sentence_start_positions[sentence_ind]
                        current_word_start_position_in_simplified_sentence = words_start_positions_in_simplified_sentence[word_ind]
                        #print(current_word)
                        suggested_word = current_word
                        if model == "bert":
                            suggested_word = bert_model(sentences[sentence_ind], current_word, current_word_start_position_in_initial_sentence, complexity_method)
                        elif model == "synonym":
                            suggested_word = synonym_model(current_word, complexity_method)

                        #print(suggested_word)
                        simplified_sentence = simplified_sentence[:current_word_start_position_in_simplified_sentence] + suggested_word + simplified_sentence[current_word_start_position_in_simplified_sentence + len(current_word):]
                        simplification_offset = len(suggested_word) - len(current_word)

                        for new_word_ind in range(word_ind + 1, len(words[sentence_ind])):
                            words_start_positions_in_simplified_sentence[new_word_ind] += simplification_offset

        simplified_sentences.append(simplified_sentence)
        print(sentences[sentence_ind])
        print(simplified_sentence)
        print()

    return simplified_sentences

In [ ]:
print(simplify(text_location="/tmp/NEREL-master/NEREL-v1.1/train/023.txt", model="synonym", complexity_method="len"))

Финансовым директором социальной сети Facebook назначен 39-летний Дэвид Эберсман (David Ebersman), сообщает The Wall Street Journal.
Финансовым директором светской сети Facebook назначен 39-летний Дэвид Эберсман (David Ebersman), пишет The Wall Street Journal.

На работу в Facebook он выйдет в сентябре.
На работу в Facebook он выйдет в сентябре.

Ранее Эберсман был финансовым директором биотехнологической компании Genentech.
Ранее Эберсман был финансовым директором биотехнологической компании Genentech.

Эберсман подчеркнул, что видит много общего между Facebook и Genentech.
Эберсман указал, что видит много общего между Facebook и Genentech.

В частности, это две быстрорастущие компании с сильной корпоративной культурой.
В частности, это две быстрорастущие компании с сильной общей культурой.

Также он заявил, что Facebook ожидает 70-процентное увеличение выручки в 2009 году.
Также он заявил, что Facebook ожидает 70-процентное увеличение выручки в 2009 году.

В компании Genentech Дэвид 

#Assessment

In [ ]:
def generate_simplifications(file_list, models):
    ind = 1
    for file_name in file_list:
        for model, complexity_method in models:
            print(ind, model, complexity_method)
            file_path = os.path.join(dataset_dir, file_name)
            if os.path.isfile(file_path):
                simplified_sentences = simplify(file_path, model, complexity_method)
                if not os.path.exists(os.path.join(simple_dir, model, complexity_method)):
                    os.makedirs(os.path.join(simple_dir, model, complexity_method))
                write_list_to_file(simplified_sentences, os.path.join(simple_dir, model, complexity_method, file_name))
            print()
        ind += 1

In [ ]:
dataset_dir = "/tmp/NEREL-master/NEREL-v1.1/train/"
file_list = os.listdir(dataset_dir)
txt_file_list = [file_name for file_name in file_list if file_name.endswith(".txt")]
random.shuffle(txt_file_list)

simple_dir = '/content/drive/My Drive/simplifications'

if not os.path.exists(os.path.join(simple_dir, 'original')):
    os.makedirs(os.path.join(simple_dir, 'original'))

models = [("baseline", ""), ("bert", "len"), ("bert", "frequency"), ("synonym", "len"), ("synonym", "frequency")]

In [ ]:
write_list_to_file(txt_file_list, os.path.join(simple_dir, 'file_list.txt'))

In [ ]:
# generating simplifications for random 20 files from the dataset using baseline model
base_sample_size = 20 #20
base_sample_file_list = txt_file_list[:base_sample_size]
print(base_sample_file_list)

generate_simplifications(base_sample_file_list, models[:1]) #[("baseline", "")]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['114858_text.txt', '505701_text.txt', '3948_text.txt', '117464_text.txt', '24989_text.txt', '55256_text.txt', '109586_text.txt', '15371_text.txt', '121454_text.txt', '506305_text.txt', '25455_text.txt', '28090_text.txt', '12146_text.txt', '141181_text.txt', '131643_text.txt', '139852_text.txt', '132624_text.txt', '527071_text.txt', '99993_text.txt', '037.txt']
1 baseline 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Федеральный закон от 03.04.2017 № 53-ФЗ 3 апреля 2017 года Президент России Владимир Путин подписал закон о ратификации соглашения между Россией и Южной Осетией о повышении пенсий российским гражданам, постоянно проживающим в республике.
3 апреля 2017 года Президент России Владимир Путин подписал закон о ратификации соглашения между Россией и Южной Осетией о повышении пенсий.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Сотомайор родилась и выросла в малообеспеченной пуэрториканской семье в Нью-Йорке, окончила Принстонский университет и школу права в Йеле.
Сотомайор родилась и выросла в Нью-Йорке, штат Нью-Йорк.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Работала прокурором, занималась частной юридической практикой, а в 1992 году стала федеральным судьей в Южном округе Нью-Йорка.
В 1992 году стала федеральным судьей в Южном округе Нью-Йорка.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


На это место ее выдвинул президент-республиканец Джордж Буш-старший, а его преемник-демократ Билл Клинтон назначил Сотомайор в федеральный апелляционный суд.  AP отмечает, что в случае благополучного утверждения Сотомайор станет самым опытным юристом, пришедшим в Верховный суд за последние 70 лет.
На это место ее выдвинул президент-республиканец Джордж Буш-старший, а его преемник-демократ Билл Клинтон назначил Сотомайор в федеральный апелляционный суд.

Кроме нее, в числе претендентов на место Саутера называли высшего чиновника министерства юстиции Елену Каган (Elena Kagan) и судью апелляционного суда Дайан Вуд (Diane Wood).
Кроме нее, в списке претендентов на место Саутера называли высшее чиновничество министерства юстиции Елену Каган (Elena Kagan) и судью апелляционного суда Дайан Вуд (Diane Wood).




In [ ]:
# generating simplifications for random 200 files from the dataset using proposed models
sample_size = 100 #200
sample_file_list = txt_file_list[:sample_size]
print(sample_file_list)

generate_simplifications(sample_file_list, models[1:])

Выходные данные были обрезаны до нескольких последних строк (5000).
Об этом объявлено утром в понедельник, 26 ноября, сообщил Regnum Центр источник в обладминистрации.

После череды отставок в администрации Брянской области, которые произвёл губернатор Николай Денин (своих постов лишились три его заместителя и трое начальников департаментов), с понедельника, 26 ноября начала появляться информация о новых назначениях.
После череды отставок в администрации Брянской области, которые произвёл губернатор Николай Денин (своих постов лишились три его заместителя и трое начальников департаментов), с понедельника, 26 ноября начала появляться информация о новых назначениях.

Самое неожиданное назначение - приход на место заместителя губернатора по экономике бывшего конкурента Николая Денина на губернаторских выборах 14 октября Андрея Пономарёва.
Самое неожиданное назначение - приход на место заместителя губернатора по экономике бывшего конкурента Николая Денина на губернаторских выборах 14 октяб

In [ ]:
def count_syllables(word):
    word = re.sub(r'[^аеёиоуыэюя]', '', word.lower())
    return len(word)

def asw(sentences): # average number of syllables in the word
    total_avg_syllables = 0
    total_sentences = 0
    for sentence in sentences:
        words = re.findall(r'\b\w+\b', sentence)
        if len(words) != 0:
            total_sentences += 1
            total_syllables = 0
            for word in words:
                total_syllables += count_syllables(word)
            total_avg_syllables += total_syllables / len(words)
    return total_avg_syllables / total_sentences

In [ ]:
bert_embed_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_embed_model = AutoModel.from_pretrained("bert-base-uncased")

def bert_cos_dist(original_sentences, simple_sentences):
    avg_dist = 0
    for i in range(min(len(simple_sentences), len(original_sentences))):
        sentence1 = original_sentences[i]
        sentence2 = simple_sentences[i]

        encoded_input = bert_embed_tokenizer([sentence1, sentence2], padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            output = bert_embed_model(**encoded_input)

        sentence1_embedding = output.last_hidden_state[0, 0, :]
        sentence2_embedding = output.last_hidden_state[1, 0, :]

        dist = cosine_distances(sentence1_embedding.reshape(1, -1), sentence2_embedding.reshape(1, -1)).item()
        avg_dist += dist    
    return avg_dist / min(len(simple_sentences), len(original_sentences))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentences1 = ["Мне нравятся коты", "I love you"]
sentences2 = ["Я люблю собак", "Je t'aime"]
distance = bert_cos_dist(sentences1, sentences2)
print("Cosine distance:", distance)

Cosine distance: 0.08052542805671692


In [ ]:
def assess_model(simple_dir, model, file_list, metric):
    avg_metric = 0
    files_num = 0
    for file_name in file_list:
        simple_location = os.path.join(simple_dir, model[0], model[1], file_name)
        simple_sentences = read_list_from_file(simple_location)
        if len(simple_sentences) != 0:
            files_num += 1
            if metric == 'asw':
                avg_metric += asw(simple_sentences)
            elif metric == 'bert_cos_dist':
                original_location = os.path.join(simple_dir, 'original', file_name)
                original_sentences = read_list_from_file(original_location)
                if len(simple_sentences) != len(original_sentences):
                    print(simple_sentences, original_sentences)
                avg_metric += bert_cos_dist(simple_sentences, original_sentences)
    return avg_metric/files_num

In [ ]:
for model in models:
    for metric in ["asw", "bert_cos_dist"]:
        if model == ("baseline", ""):
            file_list = base_sample_file_list
        else:
            file_list = sample_file_list
        res = assess_model(simple_dir, model, file_list, metric)
        print("Model: " + str(model[0]))
        print("Complexity method: " + str(model[1]))
        print("Metric: " + metric)
        print("Result: " + str(res))
        print()

Model: baseline
Complexity method: 
Metric: asw
Result: 2.4319689635569652

Model: baseline
Complexity method: 
Metric: bert_cos_dist
Result: 0.018778686074329774

Model: bert
Complexity method: len
Metric: asw
Result: 2.06198362947108

Model: bert
Complexity method: len
Metric: bert_cos_dist
Result: 0.014339098001478782

Model: bert
Complexity method: frequency
Metric: asw
Result: 2.442835099455242

Model: bert
Complexity method: frequency
Metric: bert_cos_dist
Result: 0.0011896186038506703

Model: synonym
Complexity method: len
Metric: asw
Result: 2.247671556487743

Model: synonym
Complexity method: len
Metric: bert_cos_dist
Result: 0.005315173363085831

Model: synonym
Complexity method: frequency
Metric: asw
Result: 2.461087948158949

Model: synonym
Complexity method: frequency
Metric: bert_cos_dist
Result: 0.00042582925744976714

